# Mão na massa!!!

Vamos construir um modelo de classificação, baseado no desafio Kaggle ["Titanic: Machine Learning from Disaster”](https://www.kaggle.com/c/titanic/data). Esse modelo de classificação terá como objetivo prever quais passageiros sobreviveram ao naufrágio.

**Fase 1 - Identificação do problema de negócio:**

Aqui a gente vai identificar os dados disponíveis, qual sua estrutura e como vamos utilizar para a nossa análise. O Kaggle já disponibiliza os [dados](https://www.kaggle.com/c/titanic/data) rotulados para constriução do modelo e um dado de teste para submeter na competição .

Nesta atividade a gente vai usar apenas a base para construção do modelo. 

Na célula abaixo podemos ver como importar o conjunto de dados para a variavel 'dados':

In [0]:
import pandas as pd

# Importa base de dados:
uri = 'https://drive.google.com/u/0/uc?id=1OL3yv6CdvSsgLaCwuNQtGA-Ku1GAEOpV&export=download'
dados = pd.read_csv(uri)
dados.head()

**Fase 2 - Análise exploratória e preparação dos dados**

Nessa parte a gente vai fazer algumas estatísticas e limpeza da nossa base de dados.

O primeiro passo é ter uma visão geral de como os dados estão se comportando:

In [0]:
#Análise estatistica descritiva - variáveis numéricas
dados.describe()

In [0]:
#Análise estatistica descritiva - variáveis categóricas
dados.describe(include=['O'])

Ainda nessa fase, o próximo passo é a limpeza dos dados. Vamos remover alguns atributos e registros que não são relevantes para a análise, assim como codificar as variáveis textuais em números.

In [0]:
# Remover atributos irrelevantes 
atributos_irrelevantes = ['Name', 'Ticket', 'Cabin', 'PassengerId']
dados = dados.drop(atributos_irrelevantes, axis=1)

In [0]:
# Remove registros nulos
dados = dados.dropna()

In [0]:
# Codifica as variáveis string para numérico

dados['Sex'] = dados['Sex'].map({'female':1, 'male':0})
dados['Embarked'] = dados['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

dados.describe()

Por fim, a última etapa desta fase é segmentar a base de dados em "treinamento" e "validação":


In [0]:
import sklearn.model_selection
treinamento, validacao = sklearn.model_selection.train_test_split(dados, random_state=1)
treinamento.shape, validacao.shape

In [0]:
# Separa as features do target:

treinamento_x = treinamento.drop('Survived', axis=1)
treinamento_y = treinamento['Survived']
validacao_x = validacao.drop('Survived', axis=1)
validacao_y = validacao['Survived']

**Fase 3 - Criação e otimização de modelos**

Nessa fase, com os dados já estruturados, o próximo passo que devemos tomar é a criação dos modelos de previsão. Neste exemplo vamos treinar dois modelos:


1.   Regressão Logística
2.   Rede Neural

Abaixo o código para o treinamento para o modelo de regressão:



In [0]:
#Modelo construido com o scikit-learn
from sklearn.linear_model import LogisticRegression

modelo_reglog = LogisticRegression(solver='liblinear')
modelo_reglog.fit(treinamento_x, treinamento_y)
acc_reglog = round( modelo_reglog.score(treinamento_x, treinamento_y) * 100, 2)
print ('Acurácia: ' + str(acc_reglog) + '%')


In [0]:
# Testando o modelo na base de validação:
acc_reglog = round( modelo_reglog.score(validacao_x, validacao_y) * 100, 2)
print ('Acurácia: ' + str(acc_reglog) + '%')

Abaixo o treinamento da rede neural:





In [0]:
#modelo construido com o tensorflow
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(7, activation=tf.nn.relu, input_shape=[7]),
  tf.keras.layers.Dense(14, activation=tf.nn.softmax),
  tf.keras.layers.Dense(196, activation=tf.nn.softmax),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [0]:
model.fit(treinamento_x, treinamento_y, epochs=200)
model.evaluate(validacao_x, validacao_y)

**Fase 4 - Implantação do modelo**

Por fim, a última etapa, é a aplicação no modelo na base. Para isso, devemos atentar para que os dados de saída sejam no mesmo formato que os dados de origem, no caso, binário ("0" se morreu e "1"se sobreviveu)

In [0]:
previsoes = model.predict(validacao_x)
previsoes

In [0]:
previsoes_bin=[]
for i in previsoes[:,1]:
    if i > 0.5:
        previsoes_bin.append(1)
    else:
        previsoes_bin.append(0)
previsoes_bin